In [11]:
import os
import json
import logging
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from ultralytics import YOLO
import cv2
import json

In [12]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set paths
train_dir = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\train"
val_dir = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\val"
test_dir = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\test"
train_ann_path = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\train_iou_results.json"
val_ann_path = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\val_iou_results.json"
test_ann_path = r"C:\Users\bumin\Downloads\DLCV project\TACO\dataset_split\test_iou_results.json"

In [13]:
detection_model = YOLO("yolov8n.pt") #from the previous notebook

In [25]:
import numpy as np
from tensorflow.keras.utils import Sequence

from tensorflow.keras.utils import Sequence
import os
import cv2
import numpy as np

class ObjectDetectionDataGenerator(Sequence):
    def __init__(self, annotations, image_folder, batch_size, input_size, detection_model, category_map):
        """
        Custom data generator for classification using object detection annotations.
        Args:
            annotations (list): List of annotations with image_id, detected_bbox, and supercategory.
            image_folder (str): Path to the folder containing subdirectories for classes.
            batch_size (int): Batch size.
            input_size (tuple): Input size for the classification model (e.g., (224, 224)).
            category_map (dict): Mapping from supercategory to class index.
        """
        self.annotations = annotations
        self.image_folder = image_folder
        self.batch_size = batch_size
        self.input_size = input_size
        self.detection_model = detection_model
        self.category_map = category_map
        self.image_paths = {}  # Maps image_id to the full path
        self._build_image_paths()

    def _build_image_paths(self):
        """
        Build a dictionary to map image_id to the full file path by scanning subdirectories.
        """
        for root, _, files in os.walk(self.image_folder):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Extract image_id (file name without extension)
                    image_id = os.path.splitext(file)[0]
                    full_path = os.path.join(root, file)
                    self.image_paths[image_id] = full_path

    def __len__(self):
        """
        Number of batches per epoch.
        """
        return len(self.annotations) // self.batch_size

    def __getitem__(self, idx):
        """
        Generate one batch of data.
        """
        batch_data = self.annotations[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = []
        labels = []

        for item in batch_data:
            image_id = item["image_id"]
            if image_id not in self.image_paths:
                print(f"Image ID {image_id} not found in directory. Skipping.")
                continue

            # Load image
            image_path = self.image_paths[image_id]
            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to load image: {image_path}. Skipping.")
                continue

            # Crop the detected bounding box
            x_min, y_min, x_max, y_max = map(int, item["detected_bbox"])
            cropped = image[y_min:y_max, x_min:x_max]

            # Resize and normalize the cropped image
            cropped_resized = cv2.resize(cropped, self.input_size)
            cropped_resized = cropped_resized / 255.0

            # Append to batch
            images.append(cropped_resized)
            labels.append(self.category_map[item["supercategory"]])

        # Ensure batch is not empty
        if len(images) == 0:
            return np.empty((0, *self.input_size, 3)), np.empty((0, len(self.category_map)))

        # Convert to numpy arrays
        images = np.array(images)
        labels = np.array(labels)

        # One-hot encode labels
        labels_one_hot = np.zeros((len(labels), len(self.category_map)))
        labels_one_hot[np.arange(len(labels)), labels] = 1

        return images, labels_one_hot




In [26]:

with open(train_ann_path, 'r') as f:
    train_annotations = json.load(f)


with open(val_ann_path, 'r') as f:
    val_annotations = json.load(f)

# Create a mapping from supercategory to class index
unique_supercategories = sorted(set(ann['supercategory'] for ann in train_annotations))
category_map = {supercategory: idx for idx, supercategory in enumerate(unique_supercategories)}

print("Category Map:", category_map)

Category Map: {'Aluminium foil': 0, 'Blister pack': 1, 'Bottle': 2, 'Bottle cap': 3, 'Broken glass': 4, 'Can': 5, 'Carton': 6, 'Cigarette': 7, 'Cup': 8, 'Food waste': 9, 'Glass jar': 10, 'Lid': 11, 'Other plastic': 12, 'Paper': 13, 'Paper bag': 14, 'Plastic bag & wrapper': 15, 'Plastic container': 16, 'Plastic utensils': 17, 'Rope & strings': 18, 'Shoe': 19, 'Squeezable tube': 20, 'Straw': 21, 'Styrofoam piece': 22, 'Unlabeled litter': 23}


In [27]:
# Initialize the data generators
train_generator = ObjectDetectionDataGenerator(
    annotations=train_annotations,
    image_folder=train_dir,
    detection_model=detection_model,
    batch_size=32,
    input_size=(224, 224),
    category_map=category_map
)

val_generator = ObjectDetectionDataGenerator(
    annotations=val_annotations,
    image_folder=train_dir,
    detection_model=detection_model,
    batch_size=32,
    input_size=(224, 224),
    category_map=category_map
)

In [28]:
images, labels = train_generator[0]
print(f"Images batch shape: {images.shape}")
print(f"Labels batch shape: {labels.shape}")


Image ID 1136 not found in directory. Skipping.
Image ID 260 not found in directory. Skipping.
Image ID 847 not found in directory. Skipping.
Image ID 920 not found in directory. Skipping.
Image ID 97 not found in directory. Skipping.
Image ID 976 not found in directory. Skipping.
Image ID 99 not found in directory. Skipping.
Image ID 337 not found in directory. Skipping.
Image ID 367 not found in directory. Skipping.
Image ID 873 not found in directory. Skipping.
Image ID 102 not found in directory. Skipping.
Image ID 103 not found in directory. Skipping.
Image ID 1044 not found in directory. Skipping.
Image ID 1045 not found in directory. Skipping.
Image ID 1063 not found in directory. Skipping.
Image ID 1103 not found in directory. Skipping.
Image ID 1106 not found in directory. Skipping.
Image ID 1110 not found in directory. Skipping.
Image ID 1113 not found in directory. Skipping.
Image ID 1123 not found in directory. Skipping.
Image ID 1127 not found in directory. Skipping.
Image

In [29]:
# Number of classes (from category_map)
num_classes = len(category_map)

# Load the ResNet50 base model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
base_model.trainable = False

# Add custom layers for classification
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")  # Output layer with num_classes
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,435,992 (138.99 MB)

 Trainable params: 12,848,280 (49.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [30]:
from tensorflow.keras import layers, models, optimizers
# Update the model's output layer
model.layers[-1] = layers.Dense(num_classes, activation="softmax")
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)

Image ID 1136 not found in directory. Skipping.
Image ID 260 not found in directory. Skipping.
Image ID 847 not found in directory. Skipping.
Image ID 920 not found in directory. Skipping.
Image ID 97 not found in directory. Skipping.
Image ID 976 not found in directory. Skipping.
Image ID 99 not found in directory. Skipping.
Image ID 337 not found in directory. Skipping.
Image ID 367 not found in directory. Skipping.
Image ID 873 not found in directory. Skipping.
Image ID 102 not found in directory. Skipping.
Image ID 103 not found in directory. Skipping.
Image ID 1044 not found in directory. Skipping.
Image ID 1045 not found in directory. Skipping.
Image ID 1063 not found in directory. Skipping.
Image ID 1103 not found in directory. Skipping.
Image ID 1106 not found in directory. Skipping.
Image ID 1110 not found in directory. Skipping.
Image ID 1113 not found in directory. Skipping.
Image ID 1123 not found in directory. Skipping.
Image ID 1127 not found in directory. Skipping.
Image

c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Image ID 888 not found in directory. Skipping.
Image ID 90 not found in directory. Skipping.
Image ID 103 not found in directory. Skipping.
Image ID 1193 not found in directory. Skipping.
Image ID 1202 not found in directory. Skipping.
Image ID 1208 not found in directory. Skipping.
Image ID 1213 not found in directory. Skipping.
Image ID 1216 not found in directory. Skipping.
Image ID 1260 not found in directory. Skipping.
Image ID 1278 not found in directory. Skipping.
Image ID 1288 not found in directory. Skipping.
Image ID 1294 not found in directory. Skipping.
Image ID 1353 not found in directory. Skipping.
Image ID 1448 not found in directory. Skipping.
Image ID 1481 not found in directory. Skipping.
Image ID 158 not found in directory. Skipping.
Image ID 179 not found in directory. Skipping.
Image ID 261 not found in directory. Skipping.
Image ID 296 not found in directory. Skipping.
Image ID 322 not found in directory. Skipping.
Image ID 326 not found in directory. Skipping.
Im

InvalidArgumentError: Graph execution error:

Detected at node sequential_1_1/resnet50_1/conv1_bn_1/batchnorm/mul_1 defined at (most recent call last):
  File "c:\Users\bumin\anaconda3\envs\c20599\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\runpy.py", line 86, in _run_code

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\asyncio\windows_events.py", line 321, in run_forever

  File "c:\Users\bumin\anaconda3\envs\c20599\lib\asyncio\base_events.py", line 603, in run_forever

  File "c:\

In [ ]:
print(f"Training set classes: {len(train_data.class_indices)}")
print(f"Validation set classes: {len(val_data.class_indices)}")
print(f"Class indices (train): {train_data.class_indices}")
print(f"Class indices (val): {val_data.class_indices}")


In [ ]:
# Train the model
history = model.fit(
    train_data,
    epochs=15,  # Number of epochs
    validation_data=val_data,  # Validation set
    steps_per_epoch=train_data.samples // train_data.batch_size,  # Steps per epoch
    validation_steps=val_data.samples // val_data.batch_size  # Validation steps
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data, steps=test_data.samples // test_data.batch_size)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the model
model.save("garbage_classifier.h5")
print("Model saved as garbage_classifier.h5")


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict on the test set
predictions = model.predict(test_data, steps=test_data.samples // test_data.batch_size)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# Unfreeze some layers of the base model
base_model.trainable = True

# Compile the model again with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
history_fine = model.fit(
    train_data,
    epochs=5,
    validation_data=val_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=val_data.samples // val_data.batch_size
)
